### Manipulation of PeaskDB *de novo*-assisted database search results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Search database included marine fungi and labrinthulomyces discovered using _de novo_ peptide sequencing
    XInteract file includes precursor intensities and protein descriptions

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDB search results as `csv` files into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 378/

'ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_DB search psm.csv'
'ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_de novo only peptides.csv'
 ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_peptide.csv
 ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_peptides.pep.xml
 ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_protein-peptides.csv
 ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_proteins.csv
 ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_proteins.fasta
 ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm.csv
 ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_NAAF.csv
 ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_NAAF_totals
 ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_stripped_peptides.fas
 ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_stripped_peptides.txt
 ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_totals.csv


## 378: 100 m trap filtered on 0.3 um GF-75

In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup378 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB_167_378-100m-trap_15ppm_peptide.csv")

# remove redundant rows
peaksdb378 = pd.DataFrame.drop_duplicates(peaksdbdup378)

print(peaksdb378.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb378.columns = columns

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup378))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb378))

#look at the dataframe
peaksdb378.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 378_100m_trap_SKQ17_P2', 'Fraction', 'Scan', 'Source File',
       '#Spec', '#Spec 378_100m_trap_SKQ17_P2', 'Accession', 'PTM', 'AScore'],
      dtype='object')
# redundant peaksdb peptides in combined dataframe 120
# nonredundant peaksdb peptides in combined dataframe 120


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source File,#Spec,#Spec,Accession,PTM,AScore
0,LPQVEGTGGDVQPSQDLVR,86.46,1994.0068,19,0.2,998.0109,54.03,63800000.0,4,15243,20181214_378_etnp2017_100m_trap.raw,4,4,gi|54036848,NaN,NaN
1,VIGQNEAVDAVSNAIR,67.23,1654.8638,16,0.7,828.4398,64.54,41700000.0,4,18159,20181214_378_etnp2017_100m_trap.raw,5,5,gi|54036848,NaN,NaN
2,AIQQQIENPLAQQILSGELVPGK,64.26,2473.3540,23,0.2,825.4587,86.70,774000.0,4,24111,20181214_378_etnp2017_100m_trap.raw,2,2,gi|54036848,NaN,NaN
3,AIDLIDEAASSIR,45.64,1372.7197,13,0.6,458.5808,76.55,26000000.0,4,21495,20181214_378_etnp2017_100m_trap.raw,6,6,gi|54036848,NaN,NaN
4,AIDLIDEAASSLR,45.64,1372.7197,13,0.6,458.5808,76.55,26000000.0,4,21495,20181214_378_etnp2017_100m_trap.raw,6,6,NaN,NaN,NaN


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb378['A'] = peaksdb378['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb378['C'] = peaksdb378['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb378['D'] = peaksdb378['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb378['E'] = peaksdb378['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb378['F'] = peaksdb378['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb378['G'] = peaksdb378['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb378['H'] = peaksdb378['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb378 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb378['I'] = peaksdb378['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb378['K'] = peaksdb378['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb378['L'] = peaksdb378['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb378['M'] = peaksdb378['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb378['N'] = peaksdb378['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb378['P'] = peaksdb378['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb378['Q'] = peaksdb378['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb378['R'] = peaksdb378['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb378['S'] = peaksdb378['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb378['T'] = peaksdb378['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb378['V'] = peaksdb378['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb378['W'] = peaksdb378['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb378['Y'] = peaksdb378['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb378['c-carb'] = peaksdb378['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb378['m-oxid'] = peaksdb378['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb378['n-deam'] = peaksdb378['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaksdb378['q-deam'] = peaksdb378['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaksdb378['k-hydr'] = peaksdb378['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
#peaksdb378['p-hydr'] = peaksdb378['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaksdb378['r-meth'] = peaksdb378['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb378['stripped peptide'] = peaksdb378['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb378['stripped length'] = peaksdb378['stripped peptide'].apply(len)

peaksdb378['NAAF num.'] = peaksdb378['Area'] / peaksdb378['stripped length']

# total the number of modifications in sequence
peaksdb378['ptm-total'] = peaksdb378['c-carb'] + peaksdb378['m-oxid'] + peaksdb378['n-deam'] + peaksdb378['q-deam'] + peaksdb378['k-hydr'] + peaksdb378['r-meth']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb378['stripped I-L']= peaksdb378['stripped peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb378.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm.csv")

# check out the results
peaksdb378.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,NAAF num.,ptm-total,stripped I-L
0,LPQVEGTGGDVQPSQDLVR,86.46,1994.0068,19,0.2,998.0109,54.03,63800000.0,4,15243,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,3.357895e+06,0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR,67.23,1654.8638,16,0.7,828.4398,64.54,41700000.0,4,18159,...,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,2.606250e+06,0,VLGQNEAVDAVSNALR
2,AIQQQIENPLAQQILSGELVPGK,64.26,2473.3540,23,0.2,825.4587,86.70,774000.0,4,24111,...,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,3.365217e+04,0,ALQQQLENPLAQQLLSGELVPGK
3,AIDLIDEAASSIR,45.64,1372.7197,13,0.6,458.5808,76.55,26000000.0,4,21495,...,0,0,0,0,0,AIDLIDEAASSIR,13,2.000000e+06,0,ALDLLDEAASSLR
4,AIDLIDEAASSLR,45.64,1372.7197,13,0.6,458.5808,76.55,26000000.0,4,21495,...,0,0,0,0,0,AIDLIDEAASSLR,13,2.000000e+06,0,ALDLLDEAASSLR


### Exporting txt files of stripped peptides at confidence cutoffs:

In [6]:
# keep only peptide column peptides >20 -10lgP
pep378moddup = peaksdb378[["Peptide"]]

# keep only the stripped peptide column >20 -10lgP
# this is what we'll use for UniPept input, etc
pep378dup = peaksdb378[["stripped peptide"]]

# deduplicate both of these lists
pep378mod = pep378moddup.drop_duplicates()
pep378 = pep378dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 378:", len(pep378moddup))
print("Deduplicated modified PeaksDB peptides in 378:", len(pep378mod))
print("Total stripped PeaksDB peptides in 378:", len(pep378dup))
print("Deduplicated stripped PeaksDB peptides in 378:", len(pep378))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep378.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_stripped_peptides.fas

# look at the stripped peptides
pep378.head()

Total modified PeaksDB peptides in 378: 120
Deduplicated modified PeaksDB peptides in 378: 120
Total stripped PeaksDB peptides in 378: 120
Deduplicated stripped PeaksDB peptides in 378: 120


,stripped peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,AIQQQIENPLAQQILSGELVPGK
3,AIDLIDEAASSIR
4,AIDLIDEAASSLR


## NAAF correction and exporting files with AA and PTM totals:

In [7]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb378['A'].sum(),
        'C': peaksdb378['C'].sum(),
        'D': peaksdb378['D'].sum(),
        'E': peaksdb378['E'].sum(),
        'F': peaksdb378['F'].sum(),
        'G': peaksdb378['G'].sum(),
        'H': peaksdb378['H'].sum(),
        'I': peaksdb378['I'].sum(),
        'K': peaksdb378['K'].sum(),
        'L': peaksdb378['L'].sum(),
        'M': peaksdb378['M'].sum(),
        'N': peaksdb378['N'].sum(),
        'P': peaksdb378['P'].sum(),
        'Q': peaksdb378['Q'].sum(),
        'R': peaksdb378['R'].sum(),
        'S': peaksdb378['S'].sum(),
        'T': peaksdb378['T'].sum(),
        'V': peaksdb378['V'].sum(),
        'W': peaksdb378['W'].sum(),
        'Y': peaksdb378['Y'].sum(),
        'c-carb': peaksdb378['c-carb'].sum(),
        'm-oxid': peaksdb378['m-oxid'].sum(),
        'n-deam': peaksdb378['n-deam'].sum(),
        'q-deam': peaksdb378['q-deam'].sum(),
        'k-hydr': peaksdb378['k-hydr'].sum(),
        'r-meth': peaksdb378['r-meth'].sum(),
        'Total area': peaksdb378['Area'].sum(),
        'Total length': peaksdb378['stripped length'].sum()
       }

totalpeaksdb378 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb378['% C w/ carb'] = totalpeaksdb378['c-carb'] / totalpeaksdb378['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb378['% M w/ oxid'] = totalpeaksdb378['m-oxid'] / totalpeaksdb378['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb378['% N w/ deam'] = totalpeaksdb378['n-deam'] / totalpeaksdb378['N'] 

# calculate percentage of Q's that are deamidated
totalpeaksdb378['% Q w/ deam'] = totalpeaksdb378['q-deam'] / totalpeaksdb378['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaksdb378['% K w/ hydr'] = totalpeaksdb378['k-hydr'] / totalpeaksdb378['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaksdb378['% P w/ hydr'] = totalpeaksdb378['p-hydr'] / totalpeaksdb378['P'] 

# calculate percentage of R's that are methylated
totalpeaksdb378['% R w/ meth'] = totalpeaksdb378['r-meth'] / totalpeaksdb378['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb378['NAAF denom.'] = totalpeaksdb378['Total area'] / totalpeaksdb378['Total length']

# write modified dataframe to new txt file
totalpeaksdb378.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_totals.csv")

totalpeaksdb378.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,133,3,84,140,14,91,7,65,81,96,...,6,693658100.0,1118,1.0,0.25,0.0,0.057143,0.0,0.139535,620445.527728


In [8]:
# use the calculated NAAF factor (in totalpeaksdb378 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 620445.527728

# use NAAF >20 -10lgP to get NAAF
peaksdb378['NAAF factor'] = (peaksdb378['NAAF num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb378_NAAF = peaksdb378[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', \
                                'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb378_NAAF['A-NAAF20'] = peaksdb378_NAAF['A'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['C-NAAF20'] = peaksdb378_NAAF['C'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['D-NAAF20'] = peaksdb378_NAAF['D'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['E-NAAF20'] = peaksdb378_NAAF['E'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['F-NAAF20'] = peaksdb378_NAAF['F'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['G-NAAF20'] = peaksdb378_NAAF['G'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['H-NAAF20'] = peaksdb378_NAAF['H'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['I-NAAF20'] = peaksdb378_NAAF['I'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['K-NAAF20'] = peaksdb378_NAAF['K'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['L-NAAF20'] = peaksdb378_NAAF['L'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['M-NAAF20'] = peaksdb378_NAAF['M'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['N-NAAF20'] = peaksdb378_NAAF['N'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['P-NAAF20'] = peaksdb378_NAAF['P'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['Q-NAAF20'] = peaksdb378_NAAF['Q'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['R-NAAF20'] = peaksdb378_NAAF['R'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['S-NAAF20'] = peaksdb378_NAAF['S'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['T-NAAF20'] = peaksdb378_NAAF['T'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['V-NAAF20'] = peaksdb378_NAAF['V'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['W-NAAF20'] = peaksdb378_NAAF['W'] * peaksdb378['NAAF factor']
peaksdb378_NAAF['Y-NAAF20'] = peaksdb378_NAAF['Y'] * peaksdb378['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb378_NAAF['ccarb-NAAF20'] = peaksdb378_NAAF['c-carb'] * peaksdb378_NAAF['NAAF factor']
peaksdb378_NAAF['moxid-NAAF20'] = peaksdb378_NAAF['m-oxid'] * peaksdb378_NAAF['NAAF factor']
peaksdb378_NAAF['ndeam-NAAF20'] = peaksdb378_NAAF['n-deam'] * peaksdb378_NAAF['NAAF factor']
peaksdb378_NAAF['qdeam-NAAF20'] = peaksdb378_NAAF['q-deam'] * peaksdb378_NAAF['NAAF factor']
peaksdb378_NAAF['khydr-NAAF20'] = peaksdb378_NAAF['k-hydr'] * peaksdb378_NAAF['NAAF factor']
peaksdb378_NAAF['rmeth-NAAF20'] = peaksdb378_NAAF['r-meth'] * peaksdb378_NAAF['NAAF factor']

# write the dataframe to a new csv
peaksdb378_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_NAAF.csv")

peaksdb378_NAAF.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20,khydr-NAAF20,rmeth-NAAF20
0,LPQVEGTGGDVQPSQDLVR,5.412070,0,0,2,1,0,3,0,0,...,5.41207,16.236211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,VIGQNEAVDAVSNAIR,4.200611,3,0,1,1,0,1,0,2,...,0.00000,12.601832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AIQQQIENPLAQQILSGELVPGK,0.054239,2,0,0,2,0,2,0,3,...,0.00000,0.054239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AIDLIDEAASSIR,3.223490,3,0,2,1,0,0,0,3,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AIDLIDEAASSLR,3.223490,3,0,2,1,0,0,0,2,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb378 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb378_NAAF['NAAF factor'].sum(),
        'A-NAAF': peaksdb378_NAAF['A-NAAF20'].sum(),
        'C-NAAF': peaksdb378_NAAF['C-NAAF20'].sum(),
        'D-NAAF': peaksdb378_NAAF['D-NAAF20'].sum(),
        'E-NAAF': peaksdb378_NAAF['E-NAAF20'].sum(),
        'F-NAAF': peaksdb378_NAAF['F-NAAF20'].sum(),
        'G-NAAF': peaksdb378_NAAF['G-NAAF20'].sum(),
        'H-NAAF': peaksdb378_NAAF['H-NAAF20'].sum(),
        'I-NAAF': peaksdb378_NAAF['I-NAAF20'].sum(),
        'K-NAAF': peaksdb378_NAAF['K-NAAF20'].sum(),
        'L-NAAF': peaksdb378_NAAF['L-NAAF20'].sum(),
        'M-NAAF': peaksdb378_NAAF['M-NAAF20'].sum(),
        'N-NAAF': peaksdb378_NAAF['N-NAAF20'].sum(),
        'P-NAAF': peaksdb378_NAAF['P-NAAF20'].sum(),
        'Q-NAAF': peaksdb378_NAAF['Q-NAAF20'].sum(),
        'R-NAAF': peaksdb378_NAAF['R-NAAF20'].sum(),
        'S-NAAF': peaksdb378_NAAF['S-NAAF20'].sum(),
        'T-NAAF': peaksdb378_NAAF['T-NAAF20'].sum(),
        'V-NAAF': peaksdb378_NAAF['V-NAAF20'].sum(),
        'W-NAAF': peaksdb378_NAAF['W-NAAF20'].sum(),
        'Y-NAAF': peaksdb378_NAAF['Y-NAAF20'].sum(),
        'C-carb-NAAF': peaksdb378_NAAF['ccarb-NAAF20'].sum(),
        'M-oxid-NAAF': peaksdb378_NAAF['moxid-NAAF20'].sum(),
        'N-deam-NAAF': peaksdb378_NAAF['ndeam-NAAF20'].sum(),
        'Q-deam-NAAF': peaksdb378_NAAF['qdeam-NAAF20'].sum(),
        'K-hydr-NAAF': peaksdb378_NAAF['khydr-NAAF20'].sum(),
        'R-meth-NAAF': peaksdb378_NAAF['rmeth-NAAF20'].sum()
       }

totalpeaksdb378_NAAF = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


totalpeaksdb378_NAAF['% C w/ carb. NAAF'] = totalpeaksdb378_NAAF['C-carb-NAAF'] / totalpeaksdb378_NAAF['C-NAAF']
totalpeaksdb378_NAAF['% M w/ oxid. NAAF'] = totalpeaksdb378_NAAF['M-oxid-NAAF'] / totalpeaksdb378_NAAF['M-NAAF']
totalpeaksdb378_NAAF['% N w/ deam. NAAF'] = totalpeaksdb378_NAAF['N-deam-NAAF'] / totalpeaksdb378_NAAF['N-NAAF']
totalpeaksdb378_NAAF['% Q w/ deam. NAAF'] = totalpeaksdb378_NAAF['Q-deam-NAAF'] / totalpeaksdb378_NAAF['Q-NAAF']
totalpeaksdb378_NAAF['% K w/ hydr. NAAF'] = totalpeaksdb378_NAAF['K-hydr-NAAF'] / totalpeaksdb378_NAAF['K-NAAF']
totalpeaksdb378_NAAF['% R w/ meth. NAAF'] = totalpeaksdb378_NAAF['R-meth-NAAF'] / totalpeaksdb378_NAAF['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb378_NAAF['NAAF check'] = totalpeaksdb378_NAAF['NAAF'] / 620445.527728

# write modified dataframe to new txt file, same name + totals
totalpeaksdb378_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDB-PTMopt/378/ETNP_SKQ17_PEAKSDB20_378-100m-trap_15ppm_NAAF_totals")

totalpeaksdb378_NAAF.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,110.106445,124.773942,0.248526,49.975971,60.189831,4.934141,80.032139,0.659536,52.289862,29.423688,...,0.231285,0.0,1.025997,1.0,0.393678,0.0,0.009469,0.0,0.012751,0.000177


### Using BioPython to query peptide sequences

I installed the BioPython package using `pip install biopython`. All instructions and information [here](https://www.tutorialspoint.com/biopython/index.htm). 

GitHub project: https://github.com/biopython/biopython

I'm relying on the ProtParam module to parse sequences for relative AA composition, instability, secondary structure, instability, and hydrophobicity. You can read more about that module and the studies the indecies are derived from here:

https://biopython.org/wiki/ProtParam

In [ ]:
# Bio.SeqIO is the standard Sequence Input/Output interface for BioPython 1.43 and later
# Bio.SeqIO provides a simple uniform interface to input and output assorted sequence file formats.
# (including multiple sequence alignments), but will only deal with sequences as SeqRecord objects

# for accepted file formats see https://biopython.org/wiki/SeqIO

from Bio import SeqIO
#for seq_record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/MED4/MED2_tryp_1raw_db_peptides_nmod.fasta", "fasta"):
    #print(seq_record.id)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    
# I commented the print functions out so the output doesn't take up too much space. 

In [ ]:
# seeing what the ProtParam module can do with a single protein sequence:

from Bio.SeqUtils.ProtParam import ProteinAnalysis

test_seq = "MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV"

analysed_seq = ProteinAnalysis(test_seq)
print("molecular weight of seq =", analysed_seq.molecular_weight())

# calculates the aromaticity value of a protein according to Lobry & Gautier (1994, Nucleic Acids Res., 22, 3174-3180). 
# it's simply the relative frequency of Phe+Trp+Tyr.

analysed_seq.aromaticity()
print("aromaticity of seq =", analysed_seq.aromaticity())

# secondary_structure_fraction:
# this methods returns a list of the fraction of amino acids which tend to be in helix, turn or sheet. 
# AAs in helix: V, I, Y, F, W, L
# AAs in turn: N, P, G, S
# AAs in sheet: E, M, A, L
# the returned list contains 3 values: [Helix, Turn, Sheet]

analysed_seq.secondary_structure_fraction()
print("frac in H T S =", analysed_seq.secondary_structure_fraction())

# the instability index, an implementation of the method of Guruprasad et al. (1990, Protein Engineering, 4, 155-161).
# this method tests a protein for stability. 
# any value above 40 means the protein is unstable (=has a short half life)
# NOT SURE WHAT THIS MEANS FOR PEPTIDES, BUT WE COULD DO THIS FOR PROTEINS

analysed_seq.instability_index()
print("instability =", analysed_seq.instability_index())

# count_amino_acids will do just that, and get_amino_acids_percent will return %'s for each AA across the sequence. 
analysed_seq.get_amino_acids_percent()

# taking the returned dictionary and converting to a dataframe

aadict = analysed_seq.get_amino_acids_percent()
aadf = pd.DataFrame(list(aadict.items()),columns = ['residue','% occurance']) 

aadf.head()

In [ ]:
# use SeqIO and a loop to apply count_amino_acids to each sequence in the file
# aatot will give us the total number of each residue in the entire sample output

import collections
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

all_aas = collections.defaultdict(int)
for record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas", "fasta"):
    x = ProteinAnalysis(str(record.seq))
    #print(record.id, x.count_amino_acids())
    for aa, count in x.count_amino_acids().items():
        all_aas[aa] += count        
        
# made a dataframe for amino acid total counts        
data = (all_aas)
aatot = pd.DataFrame(data, index = ['sample sequence total'])
aatot.head()

In [ ]:
from pandas import Series, DataFrame

with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas') as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2)).set_index(['ID'])

idseq.head()

In [ ]:
from pandas import Series, DataFrame



with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    aa = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        aa.count_amino_acids(seq_record.seq)
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')
s3 = Series(aa, name='AAs')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2, AAs=s3)).set_index(['ID'])

idseq.head()

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

handle = open("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas") 
for record in SeqIO.parse(handle, "fasta"): 
    seq = str(record.seq)
    X = ProtParam.ProteinAnalysis(seq)
    print(X.count_amino_acids()) 
    #print X.get_amino_acids_percent() 
    #print X.molecular_weight() 
    #print X.aromaticity() 
    #print X.instability_index() 
    #print X.flexibility() 
    #print X.isoelectric_point() 
    #print X.secondary_structure_fraction()
    
# made a data series from the count_amino_acids function
# aacount = {X.count_amino_acids()}

# made a pandas dataframe from the series generated above
# aacount = pd.DataFrame(list(data.items()),columns = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']) 

aacount = pd.DataFrame(X.count_amino_acids(), index=[0])

# look at new dataframe

# aacount.head()